In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import itertools
from imblearn.under_sampling import RandomUnderSampler
from sklearn.preprocessing import OneHotEncoder
cat_encoder = OneHotEncoder()
from mlxtend.frequent_patterns import apriori, fpmax, fpgrowth

In [2]:
#Reading the data in from Excel

print("loading data")

stroke_dataframe = pd.read_csv("Stroke_Dataset.csv")

loading data


In [3]:

targets = stroke_dataframe["Stroke"]
del stroke_dataframe["id"]

under = RandomUnderSampler(sampling_strategy=1)
stroke_dataframe, k = under.fit_resample(stroke_dataframe, targets)

#del stroke_dataframe["id"]

In [4]:
summary_df = pd.DataFrame()                                         # instantiate dataframe for concatenate step later
models = ['apriori', 'fpmax', 'fpgrowth']                           # define list of model names to add to temp dataframe
param_list = np.linspace(.1, .9, 8)                                 # create range of values to try for min_support
min_support = .1                                                    # set an initial value for min_support
i = 0                                                               # set inital value, use to iterate through 'models'

model_set = [                                                       # Set up list of models to run, min_support will come from param_list
    apriori(stroke_dataframe, min_support, max_len = 3, use_colnames=True),
    fpmax(stroke_dataframe, min_support, max_len = 3, use_colnames=True), 
    fpgrowth(stroke_dataframe, min_support, max_len = 3, use_colnames=True)
    ]

for model in model_set:
    for k in range(len(param_list)):
        min_support = param_list[k]                                 # pull the next value for min_support from the parameter grid
        model_df = pd.DataFrame(model)                              # run the model and dump the results into a temporary df
        model_df['model'] = models[i]                               # add a column with model i's name
        summary_df = pd.concat([summary_df, model_df], axis=0)      # append the temp dataframe onto the full one
    i = i + 1                                                       # increase i to pull next model's name

summary_df.reset_index(inplace=True, drop=True)                     # reset to get rid of duplicate index values


/Users/yutasugiyama/opt/anaconda3/lib/python3.9/site-packages/mlxtend/frequent_patterns/fpcommon.py:111: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
/Users/yutasugiyama/opt/anaconda3/lib/python3.9/site-packages/mlxtend/frequent_patterns/fpcommon.py:111: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
/Users/yutasugiyama/opt/anaconda3/lib/python3.9/site-packages/mlxtend/frequent_patterns/fpcommon.py:111: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


In [5]:
stroke_dataframe

,Young,Middle Aged,Old,Hypertension,Heart Disease,Married,Diabetic,Not Diabetic,Underweight,Normal Weight,Overweight,Obese,Stroke,Male,Female,Employed,Never Smoked,Smokes or Smoked,Urban,Rural
0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,1,1,0,0,1,0
1,1,0,0,0,0,0,0,1,0,0,1,0,0,1,0,1,0,0,1,0
2,0,1,0,0,0,1,0,1,0,0,0,1,0,1,0,1,0,0,1,0
3,0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,1,1,0,0,1
4,0,1,0,0,0,0,0,1,0,0,1,0,0,0,1,1,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,0,0,1,0,0,1,0,1,0,1,0,0,1,0,1,1,0,1,1,0
414,0,0,1,0,1,1,0,1,0,0,0,1,1,0,1,1,0,1,0,1
415,0,1,0,1,0,1,0,1,0,0,0,1,1,0,1,1,1,0,0,1
416,0,0,1,0,0,1,0,1,0,1,0,0,1,1,0,1,0,0,1,0


In [10]:
from mlxtend.frequent_patterns import association_rules

assoc_df = association_rules(summary_df, metric="confidence", min_threshold=0.7)
assoc_df1 = assoc_df[
    (assoc_df['lift'] > .75) &
    (assoc_df['conviction'] > 1)]
assoc_df1.to_csv("assoc_df1.csv")
